In [200]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.cross_validation import train_test_split
from dbanalysis import stop_tools
from dbanalysis import headers as hd
import time
from sklearn.metrics import mean_absolute_error
pd.set_option('display.max_columns', 500)

In [2]:
df_stop=pd.read_csv('/home/student/data/stops/10/12.csv',names=hd.get_stop_link_headers())

In [3]:
df_stop.shape

(30376, 10)

In [4]:
df_stop.columns

Index(['index', 'dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid'],
      dtype='object')

In [5]:
df_stop.head(10)

,index,dayofservice,tripid,plannedtime_arr_from,plannedtime_dep_from,actualtime_arr_from,actualtime_dep_from,plannedtime_arr_to,actualtime_arr_to,routeid
0,21,15-JAN-17 00:00:00,4097771,80579,80579,80214,80225,80689,80299,1_40
1,62,15-JAN-17 00:00:00,4097772,73317,73317,73156,73176,73426,73245,1_40
2,103,15-JAN-17 00:00:00,4097773,66297,66297,66444,66481,66461,66603,1_40
3,144,15-JAN-17 00:00:00,4097774,59535,59535,59455,59513,59684,59580,1_40
4,185,15-JAN-17 00:00:00,4097775,52241,52241,52151,52182,52428,52303,1_40
5,226,15-JAN-17 00:00:00,4097776,45041,45041,44534,44549,45228,44672,1_40
6,264,15-JAN-17 00:00:00,4097777,37650,37650,37065,37081,37804,37150,1_40
7,284,03-JAN-17 00:00:00,4097783,82582,82582,82281,82281,82695,82352,1_40
8,317,04-JAN-17 00:00:00,4097783,82582,82582,82338,82338,82695,82390,1_40
9,358,05-JAN-17 00:00:00,4097783,82582,82582,82241,82241,82695,82329,1_40


In [6]:
df_stop.loc[df_stop['tripid'] == 4097783]

,index,dayofservice,tripid,plannedtime_arr_from,plannedtime_dep_from,actualtime_arr_from,actualtime_dep_from,plannedtime_arr_to,actualtime_arr_to,routeid
7,284,03-JAN-17 00:00:00,4097783,82582,82582,82281,82281,82695,82352,1_40
8,317,04-JAN-17 00:00:00,4097783,82582,82582,82338,82338,82695,82390,1_40
9,358,05-JAN-17 00:00:00,4097783,82582,82582,82241,82241,82695,82329,1_40
10,399,06-JAN-17 00:00:00,4097783,82582,82582,82375,82385,82695,82472,1_40
11,440,09-JAN-17 00:00:00,4097783,82582,82582,82263,82263,82695,82320,1_40
12,481,10-JAN-17 00:00:00,4097783,82582,82582,82269,82281,82695,82351,1_40
13,522,13-JAN-17 00:00:00,4097783,82582,82582,82203,82203,82695,82288,1_40
14,563,16-JAN-17 00:00:00,4097783,82582,82582,82191,82191,82695,82316,1_40
15,604,17-JAN-17 00:00:00,4097783,82582,82582,82260,82260,82695,82357,1_40
16,645,18-JAN-17 00:00:00,4097783,82582,82582,82388,82401,82695,82467,1_40


In [45]:
df_stop['traveltime']=df_stop['actualtime_arr_to']-df_stop['actualtime_dep_from']

In [203]:
df_stop.describe().T

,count,mean,std,min,25%,50%,75%,max
index,30361.0,1.831337e+05,168759.172301,5.0,57328.0,126855.0,250374.0,681912.0
tripid,30361.0,3.407094e+06,697567.723628,2782369.0,2900899.0,3028734.0,4097399.0,5117333.0
plannedtime_arr_from,30361.0,5.534779e+04,16441.464264,24962.0,41272.0,55248.0,68721.0,85316.0
plannedtime_dep_from,30361.0,5.534779e+04,16441.464264,24962.0,41272.0,55248.0,68721.0,85316.0
actualtime_arr_from,30361.0,5.546021e+04,16485.443125,24679.0,41220.0,55463.0,68882.0,86156.0
actualtime_dep_from,30361.0,5.549442e+04,16488.064186,24698.0,41237.0,55509.0,68957.0,86169.0
plannedtime_arr_to,30361.0,5.548146e+04,16433.434418,25045.0,41404.0,55413.0,68851.0,85409.0
actualtime_arr_to,30361.0,5.561492e+04,16480.776898,24783.0,41362.0,55650.0,69067.0,86240.0
day,30361.0,2.707190e+00,1.903043,0.0,1.0,3.0,4.0,6.0
hour,30361.0,1.492695e+01,4.584458,6.0,11.0,15.0,19.0,23.0


In [9]:
df_stop.describe().T

,count,mean,std,min,25%,50%,75%,max
index,30376.0,1.831069e+05,168732.196196,5.0,57336.5,126853.0,250261.75,681912.0
tripid,30376.0,3.407624e+06,697855.433221,2782369.0,2900899.0,3028734.0,4097405.25,5117333.0
plannedtime_arr_from,30376.0,5.534131e+04,16441.930718,24962.0,41251.5,55248.0,68709.00,85316.0
plannedtime_dep_from,30376.0,5.534131e+04,16441.930718,24962.0,41251.5,55248.0,68709.00,85316.0
actualtime_arr_from,30376.0,5.545384e+04,16485.989735,24679.0,41210.5,55441.5,68875.75,86358.0
actualtime_dep_from,30376.0,5.548804e+04,16488.617509,24698.0,41224.5,55503.5,68950.25,86358.0
plannedtime_arr_to,30376.0,5.547498e+04,16433.905838,25045.0,41385.0,55400.0,68842.00,85409.0
actualtime_arr_to,30376.0,5.560853e+04,16481.334511,24783.0,41349.0,55639.5,69062.00,86451.0


In [10]:
df_stop['routeid'].unique()

array(['1_40', '44_36', '44_35', '11_42', '16_24'], dtype=object)

In [11]:
time.time()

1532201132.3862088

In [12]:
time_format = "%d-%b-%y %H:%M:%S"
df_stop['dt']=pd.to_datetime(df_stop['dayofservice'],format=time_format)

In [13]:
df_stop['day']=df_stop['dt'].dt.dayofweek

In [14]:
df_stop['hour']=df_stop['actualtime_arr_to']//3600

In [15]:
df_stop['hour'].dtypes

dtype('int64')

In [16]:
df_stop.head(5)

,index,dayofservice,tripid,plannedtime_arr_from,plannedtime_dep_from,actualtime_arr_from,actualtime_dep_from,plannedtime_arr_to,actualtime_arr_to,routeid,dt,day,hour
0,21,15-JAN-17 00:00:00,4097771,80579,80579,80214,80225,80689,80299,1_40,2017-01-15,6,22
1,62,15-JAN-17 00:00:00,4097772,73317,73317,73156,73176,73426,73245,1_40,2017-01-15,6,20
2,103,15-JAN-17 00:00:00,4097773,66297,66297,66444,66481,66461,66603,1_40,2017-01-15,6,18
3,144,15-JAN-17 00:00:00,4097774,59535,59535,59455,59513,59684,59580,1_40,2017-01-15,6,16
4,185,15-JAN-17 00:00:00,4097775,52241,52241,52151,52182,52428,52303,1_40,2017-01-15,6,14


In [17]:
weather = pd.read_csv('/home/student/dbanalysis/dbanalysis/resources/cleanweather.csv')

In [18]:
weather.head(10)

,date,dewpt,msl,rain,rhum,temp,vappr,wetb
0,2016-01-01 00:00:00,-0.2,1016.2,0.0,90,1.2,6.0,0.7
1,2016-01-01 01:00:00,-0.7,1016.2,0.0,85,1.5,5.8,0.8
2,2016-01-01 02:00:00,-1.3,1016.4,0.0,92,-0.2,5.6,-0.6
3,2016-01-01 03:00:00,-1.4,1016.3,0.0,95,-0.7,5.5,-1.0
4,2016-01-01 04:00:00,-1.2,1016.2,0.0,94,-0.3,5.6,-0.6
5,2016-01-01 05:00:00,-0.3,1015.4,0.0,82,2.4,6.0,1.4
6,2016-01-01 06:00:00,0.4,1014.6,0.0,83,2.9,6.3,1.9
7,2016-01-01 07:00:00,0.7,1014.0,0.0,88,2.5,6.4,1.8
8,2016-01-01 08:00:00,1.9,1013.0,0.0,80,4.9,7.0,3.7
9,2016-01-01 09:00:00,3.3,1011.9,0.0,81,6.3,7.7,5.0


In [19]:
weather['dt']=pd.to_datetime(weather['date'])
weather['hour']=weather['dt'].dt.hour
weather['date']=weather['dt'].dt.date

In [21]:
weather['hour'].dtypes

dtype('int64')

In [22]:
df_stop['date']=df_stop['dt'].dt.date


In [24]:
df_stop = pd.merge(df_stop,weather, on=['date','hour'])

In [26]:
df_stop.shape

(30375, 22)

In [75]:
df_stop['rain'].unique()

array([ 0. ,  0.2,  0.1,  1. ,  0.4,  1.6,  1.5,  1.7,  0.3,  0.8,  1.4,
        2. ,  1.8,  0.7,  1.1,  0.6,  0.5,  1.2,  2.3,  nan,  0.9,  2.8,
        2.1,  1.3,  2.6,  3.2,  1.9,  3.4,  4.2,  2.7,  3.3,  2.4,  2.2,
        4.8, 17.4,  3.8,  4.9])

In [76]:
df_stop[df_stop['rain'].isnull()]

,index,dayofservice,tripid,plannedtime_arr_from,plannedtime_dep_from,actualtime_arr_from,actualtime_dep_from,plannedtime_arr_to,actualtime_arr_to,routeid,dt_x,day,hour,date,dewpt,msl,rain,rhum,temp,vappr,wetb,dt_y,traveltime
4594,81189,11-APR-17 00:00:00,4579063,42675,42675,42798,42811,42817,42904,1_40,2017-04-11,1,11,2017-04-11,4.6,1028.7,NaN,65,10.7,8.5,7.9,2017-04-11 11:00:00,93
4595,86288,11-APR-17 00:00:00,4579611,40275,40275,40407,40429,40417,40560,1_40,2017-04-11,1,11,2017-04-11,4.6,1028.7,NaN,65,10.7,8.5,7.9,2017-04-11 11:00:00,131
4596,88202,11-APR-17 00:00:00,4581634,41475,41475,41374,41408,41617,41531,1_40,2017-04-11,1,11,2017-04-11,4.6,1028.7,NaN,65,10.7,8.5,7.9,2017-04-11 11:00:00,123
4597,55305,11-APR-17 00:00:00,4586703,42028,42028,42057,42088,42133,42189,44_36,2017-04-11,1,11,2017-04-11,4.6,1028.7,NaN,65,10.7,8.5,7.9,2017-04-11 11:00:00,101
4598,138872,11-APR-17 00:00:00,4577191,41079,41079,41454,41501,41220,41622,11_42,2017-04-11,1,11,2017-04-11,4.6,1028.7,NaN,65,10.7,8.5,7.9,2017-04-11 11:00:00,121
4599,160066,11-APR-17 00:00:00,4591168,42279,42279,42656,42672,42420,42800,11_42,2017-04-11,1,11,2017-04-11,4.6,1028.7,NaN,65,10.7,8.5,7.9,2017-04-11 11:00:00,128
4920,83571,11-APR-17 00:00:00,4579312,39075,39075,38840,38859,39217,38971,1_40,2017-04-11,1,10,2017-04-11,5.3,1028.8,NaN,68,11.0,8.9,8.4,2017-04-11 10:00:00,112
4921,90654,11-APR-17 00:00:00,4582773,36902,36902,36573,36573,37022,36709,1_40,2017-04-11,1,10,2017-04-11,5.3,1028.8,NaN,68,11.0,8.9,8.4,2017-04-11 10:00:00,136
4922,92608,11-APR-17 00:00:00,4585397,37875,37875,37774,37774,38017,37920,1_40,2017-04-11,1,10,2017-04-11,5.3,1028.8,NaN,68,11.0,8.9,8.4,2017-04-11 10:00:00,146
4923,52145,11-APR-17 00:00:00,4586031,38428,38428,39473,39473,38533,39555,44_36,2017-04-11,1,10,2017-04-11,5.3,1028.8,NaN,68,11.0,8.9,8.4,2017-04-11 10:00:00,82


In [78]:
df_stop.shape

(30375, 23)

In [79]:
df_stop = df_stop[np.isfinite(df_stop['rain'])]

In [80]:
df_stop.shape

(30361, 23)

In [83]:
df_traveltime=df_stop[['traveltime']]

In [84]:
df=df_stop[['day','hour', 'rain', 'temp']]

In [85]:
x_train, x_test, y_train, y_test=train_test_split(df, df_traveltime, test_size=0.2, random_state=4)

In [181]:
x_test.shape

(6073, 4)

In [60]:

x_train.columns

Index(['day', 'hour', 'rain', 'temp'], dtype='object')

In [54]:
y_test.describe().T

,count,mean,std,min,25%,50%,75%,max
traveltime,6075.0,120.089053,43.434627,38.0,87.0,114.0,148.0,516.0


In [170]:
nn=MLPRegressor(activation='relu',solver='adam',hidden_layer_sizes=(10),random_state=1, max_iter=1000)

In [171]:
#y_train = column_or_1d(y_train, warn=True)

In [172]:
nn.fit(x_train,y_train.values.ravel())

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=10, learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [173]:
pred_train=nn.predict(x_train)

In [174]:
pred_train.shape

(24288,)

In [175]:
pred_train

array([131.17585315, 100.37410228,  85.90711875, ..., 112.02432428,
       146.6995435 , 137.3176173 ])

In [176]:
type(pred_train)

numpy.ndarray

In [177]:
for i in range(24288):
    if pred_train[i].isnan():
        print('i value is', i)
        print('pred[i]', pred_train[i])

AttributeError: 'numpy.float64' object has no attribute 'isnan'

In [178]:
#predicting on train data 
for i in range(24288):
    print(y_train.iloc[i])
    print(pred_train[i])

traveltime    99
Name: 17351, dtype: int64
131.17585315326923
traveltime    56
Name: 3783, dtype: int64
100.37410227726386
traveltime    75
Name: 28701, dtype: int64
85.90711874729774
traveltime    143
Name: 6026, dtype: int64
134.0094672166503
traveltime    66
Name: 3204, dtype: int64
111.18057679154781
traveltime    118
Name: 23686, dtype: int64
136.37583951342074
traveltime    92
Name: 28709, dtype: int64
85.90711874729774
traveltime    171
Name: 1343, dtype: int64
142.34833940122397
traveltime    86
Name: 11548, dtype: int64
114.60292249326365
traveltime    54
Name: 23779, dtype: int64
77.57852725901411
traveltime    76
Name: 12431, dtype: int64
87.64794639999825
traveltime    143
Name: 15337, dtype: int64
100.01052816077733
traveltime    100
Name: 3094, dtype: int64
125.71302214414415
traveltime    124
Name: 16811, dtype: int64
99.55505040406847
traveltime    132
Name: 17185, dtype: int64
101.01925401034134
traveltime    184
Name: 12175, dtype: int64
126.06460795803498
traveltime 

Name: 16353, dtype: int64
136.29571296948893
traveltime    189
Name: 13080, dtype: int64
142.25099135918109
traveltime    111
Name: 3166, dtype: int64
118.57879267174896
traveltime    124
Name: 28790, dtype: int64
120.66480257641182
traveltime    98
Name: 9492, dtype: int64
105.08199730998957
traveltime    147
Name: 9702, dtype: int64
143.93110754532435
traveltime    58
Name: 23185, dtype: int64
110.12202573123629
traveltime    163
Name: 29003, dtype: int64
131.48085329337482
traveltime    58
Name: 7509, dtype: int64
105.14339505933154
traveltime    115
Name: 6373, dtype: int64
131.68886247592923
traveltime    173
Name: 14261, dtype: int64
139.99110476339786
traveltime    173
Name: 26686, dtype: int64
139.96565053188607
traveltime    100
Name: 4024, dtype: int64
135.71938188496532
traveltime    123
Name: 3753, dtype: int64
116.44244904407194
traveltime    75
Name: 3820, dtype: int64
76.81579090913692
traveltime    139
Name: 25138, dtype: int64
138.66366927725215
traveltime    144
Name:

Name: 23977, dtype: int64
128.05182956021434
traveltime    94
Name: 4323, dtype: int64
138.4213011068153
traveltime    99
Name: 7689, dtype: int64
91.59994017259798
traveltime    90
Name: 25673, dtype: int64
111.79723651557146
traveltime    154
Name: 16139, dtype: int64
120.73233595658077
traveltime    65
Name: 7899, dtype: int64
111.50751677809248
traveltime    80
Name: 19586, dtype: int64
101.39273526715456
traveltime    112
Name: 17183, dtype: int64
101.01925401034134
traveltime    103
Name: 23341, dtype: int64
88.6309912729849
traveltime    133
Name: 155, dtype: int64
121.94559649300344
traveltime    191
Name: 12702, dtype: int64
143.7032439747766
traveltime    88
Name: 24224, dtype: int64
81.52556746145052
traveltime    186
Name: 17603, dtype: int64
132.6695513029698
traveltime    109
Name: 4502, dtype: int64
134.75565031321665
traveltime    116
Name: 5856, dtype: int64
129.2861895381372
traveltime    168
Name: 19059, dtype: int64
125.68387325245108
traveltime    114
Name: 25301, 

Name: 28692, dtype: int64
85.29260156096097
traveltime    158
Name: 10332, dtype: int64
109.84680202259597
traveltime    189
Name: 4676, dtype: int64
138.91557674031688
traveltime    116
Name: 15328, dtype: int64
101.50536631379093
traveltime    93
Name: 24088, dtype: int64
109.66005766195674
traveltime    102
Name: 15644, dtype: int64
109.75196130217894
traveltime    187
Name: 16126, dtype: int64
120.64538588652182
traveltime    99
Name: 24852, dtype: int64
107.21280918269723
traveltime    151
Name: 22927, dtype: int64
144.85315997237566
traveltime    145
Name: 25204, dtype: int64
116.63803461739221
traveltime    96
Name: 21865, dtype: int64
105.06514278078282
traveltime    155
Name: 12693, dtype: int64
134.6392358743122
traveltime    156
Name: 26840, dtype: int64
126.89342628116839
traveltime    99
Name: 6550, dtype: int64
136.479708374097
traveltime    85
Name: 25244, dtype: int64
111.19453409096171
traveltime    127
Name: 10361, dtype: int64
123.80733118402341
traveltime    151
Nam

traveltime    135
Name: 1751, dtype: int64
106.26170323034042
traveltime    73
Name: 8220, dtype: int64
132.79417307793432
traveltime    59
Name: 20245, dtype: int64
100.4362660032525
traveltime    265
Name: 26967, dtype: int64
123.85754414523161
traveltime    119
Name: 3792, dtype: int64
96.54960026179043
traveltime    141
Name: 18260, dtype: int64
133.29330892168377
traveltime    65
Name: 3136, dtype: int64
128.98728815017665
traveltime    92
Name: 21628, dtype: int64
121.15717916654883
traveltime    73
Name: 15751, dtype: int64
81.12640522805991
traveltime    107
Name: 419, dtype: int64
131.2197514337796
traveltime    130
Name: 22851, dtype: int64
142.82676527541201
traveltime    117
Name: 22453, dtype: int64
122.7672081204612
traveltime    137
Name: 5791, dtype: int64
133.75755975358553
traveltime    140
Name: 19156, dtype: int64
126.32318120274088
traveltime    128
Name: 18832, dtype: int64
130.44927965576804
traveltime    136
Name: 27214, dtype: int64
128.4530723554563
traveltime

...

137.1219962612469
traveltime    102
Name: 10880, dtype: int64
138.1819416254934
traveltime    110
Name: 4118, dtype: int64
109.74700773201567
traveltime    66
Name: 8834, dtype: int64
123.210218982581
traveltime    273
Name: 27831, dtype: int64
137.97062927037123
traveltime    165
Name: 1611, dtype: int64
123.63257416321083
traveltime    111
Name: 15927, dtype: int64
139.7314729894728
traveltime    85
Name: 14020, dtype: int64
118.38781138363105
traveltime    187
Name: 29915, dtype: int64
130.98395123321956
traveltime    66
Name: 8979, dtype: int64
89.81857083283015
traveltime    83
Name: 6022, dtype: int64
109.70980554875673
traveltime    66
Name: 383, dtype: int64
120.64209695670644
traveltime    157
Name: 4444, dtype: int64
129.61671191174113
traveltime    164
Name: 6136, dtype: int64
125.56137280745048
traveltime    87
Name: 1733, dtype: int64
112.22348971377562
traveltime    99
Name: 6779, dtype: int64
141.7749533672492
traveltime    95
Name: 24940, dtype: int64
115.62354492468238

Name: 3209, dtype: int64
109.32216452204763
traveltime    93
Name: 26164, dtype: int64
133.76101304011732
traveltime    122
Name: 22261, dtype: int64
137.5891000498434
traveltime    110
Name: 4723, dtype: int64
138.43102760298495
traveltime    139
Name: 26772, dtype: int64
114.64073476767788
traveltime    142
Name: 9829, dtype: int64
124.3260170596656
traveltime    97
Name: 15715, dtype: int64
79.54461700498737
traveltime    112
Name: 18153, dtype: int64
121.93996519045966
traveltime    157
Name: 24058, dtype: int64
135.10504186731188
traveltime    97
Name: 23501, dtype: int64
117.45612846379927
traveltime    109
Name: 26542, dtype: int64
97.15916387796386
traveltime    170
Name: 21194, dtype: int64
114.39836659724104
traveltime    104
Name: 13518, dtype: int64
110.22816646530194
traveltime    80
Name: 3214, dtype: int64
108.22738323493176
traveltime    72
Name: 8616, dtype: int64
121.05828906973301
traveltime    64
Name: 26936, dtype: int64
138.09652759941494
traveltime    140
Name: 1

Name: 17734, dtype: int64
130.7502954955325
traveltime    192
Name: 19360, dtype: int64
139.21840285342287
traveltime    171
Name: 8254, dtype: int64
141.42944127477506
traveltime    91
Name: 19838, dtype: int64
139.58582434255743
traveltime    104
Name: 21545, dtype: int64
117.48676301054003
traveltime    92
Name: 25540, dtype: int64
125.12053474708718
traveltime    97
Name: 6073, dtype: int64
87.39204975998474
traveltime    56
Name: 19566, dtype: int64
99.48296104449945
traveltime    135
Name: 23311, dtype: int64
119.35508242036683
traveltime    62
Name: 26423, dtype: int64
86.1274272704072
traveltime    116
Name: 10495, dtype: int64
140.9747347253453
traveltime    99
Name: 18474, dtype: int64
142.10098842431177
traveltime    107
Name: 19702, dtype: int64
136.53143831677156
traveltime    98
Name: 28104, dtype: int64
123.68787326105668
traveltime    55
Name: 959, dtype: int64
79.96946021495543
traveltime    75
Name: 12056, dtype: int64
103.1160279451083
traveltime    155
Name: 4105, d

131.86859867669733
traveltime    125
Name: 3289, dtype: int64
137.24990524860965
traveltime    146
Name: 17533, dtype: int64
141.57222930693234
traveltime    151
Name: 19159, dtype: int64
126.32318120274088
traveltime    112
Name: 7400, dtype: int64
101.46767893013565
traveltime    110
Name: 18483, dtype: int64
123.94995825340212
traveltime    164
Name: 1390, dtype: int64
143.85299177719955
traveltime    101
Name: 27780, dtype: int64
125.15489373496963
traveltime    147
Name: 24355, dtype: int64
135.75719415937954
traveltime    182
Name: 25847, dtype: int64
130.85723205215712
traveltime    142
Name: 18469, dtype: int64
138.56144162581003
traveltime    108
Name: 15632, dtype: int64
110.43270115216055
traveltime    51
Name: 5443, dtype: int64
106.70254129070372
traveltime    107
Name: 10856, dtype: int64
109.72658534135994
traveltime    183
Name: 2892, dtype: int64
136.58686310120876
traveltime    121
Name: 19322, dtype: int64
149.86729661946393
traveltime    78
Name: 29226, dtype: int64

Name: 23107, dtype: int64
107.12911791928072
traveltime    52
Name: 19595, dtype: int64
99.19642985774523
traveltime    100
Name: 8215, dtype: int64
137.6277664432068
traveltime    167
Name: 17500, dtype: int64
147.23598001657697
traveltime    136
Name: 19359, dtype: int64
139.21840285342287
traveltime    163
Name: 23085, dtype: int64
118.0840783599492
traveltime    116
Name: 24050, dtype: int64
134.81862717898284
traveltime    63
Name: 13590, dtype: int64
77.46094264221442
traveltime    65
Name: 25668, dtype: int64
85.79944127082469
traveltime    107
Name: 29074, dtype: int64
97.64617081392053
traveltime    145
Name: 9900, dtype: int64
111.28422719291147
traveltime    125
Name: 13678, dtype: int64
117.77990508598378
traveltime    150
Name: 15861, dtype: int64
127.66477396908671
traveltime    157
Name: 28555, dtype: int64
125.63063409261831
traveltime    200
Name: 28936, dtype: int64
138.58155097987097
traveltime    135
Name: 8649, dtype: int64
130.40377006573596
traveltime    64
Name:

In [180]:
pred_test=nn.predict(x_test)

In [185]:
y_test.iloc[1]['traveltime']

116

In [198]:
#predicting on test data
for i in range(6073):
    print(y_test.iloc[i]['traveltime'], pred_test[i])
    print('error percentage', abs(y_test.iloc[i]['traveltime']- pred_test[i])*100/y_test.iloc[i]['traveltime'])
    

129 126.90905157989243
error percentage 1.6208902481453997
116 125.68732653898289
error percentage 8.351143568088697
184 136.28008767076486
error percentage 25.934734961540837
76 140.86785957906875
error percentage 85.35244681456415
88 126.49909917981911
error percentage 43.748976340703535
111 124.67624340019206
error percentage 12.32094000017303
122 99.7645386610904
error percentage 18.225787982712788
107 107.04386261274267
error percentage 0.04099309602118837
132 134.062157838346
error percentage 1.5622407866257546
110 114.53069091808545
error percentage 4.118809925532232
115 123.05447018899524
error percentage 7.003887120865426
152 138.5565505123224
error percentage 8.844374662945798
84 139.59878297111703
error percentage 66.1890273465679
57 106.1166709135522
error percentage 86.16959809395122
140 140.14041017166076
error percentage 0.10029297975768665
129 124.10945160829637
error percentage 3.7911227842663813
109 119.52860631166791
error percentage 9.659271845566893
60 104.22505517

error percentage 55.32449661323269
91 107.47365939287404
error percentage 18.10292240975169
183 140.0106675525255
error percentage 23.49143849588771
87 134.45984456964152
error percentage 54.551545482346576
144 138.58155097987097
error percentage 3.7628118195340496
114 120.33812729335344
error percentage 5.559760783643366
92 117.96296817366299
error percentage 28.22061758006846
89 116.62130275435307
error percentage 31.03517163410457
115 110.02858534860663
error percentage 4.322969262081195
102 136.53143831677156
error percentage 33.85435129095251
104 125.88313100257987
error percentage 21.04147211786526
64 111.01514136941493
error percentage 73.46115838971083
297 124.59418794917003
error percentage 58.049094966609424
185 130.16155693465075
error percentage 29.642401656945545
119 115.34197158316326
error percentage 3.0739734595266746
185 111.10353386181546
error percentage 39.94403575037002
133 119.4557755716598
error percentage 10.183627389729475
84 145.1785594329559
error percentage 

error percentage 11.22622356133767
224 147.70879140578867
error percentage 34.058575265272914
177 124.31746079085077
error percentage 29.76414644584702
73 114.08710420503463
error percentage 56.28370439045841
179 136.42652691126918
error percentage 23.784063178061913
183 123.5521991090286
error percentage 32.48513709889147
93 128.700676803132
error percentage 38.38782451949678
114 138.1184696146603
error percentage 21.156552293561678
82 81.40302927448757
error percentage 0.7280130798932132
188 129.28309389375178
error percentage 31.232396865025645
97 118.84328914033983
error percentage 22.5188547838555
95 97.24611394802277
error percentage 2.364330471602913
178 141.5510658098689
error percentage 20.47692932029837
93 125.32509064310979
error percentage 34.758161981838484
179 137.1219962612469
error percentage 23.39553281494587
82 113.71516036644086
error percentage 38.67702483712299
97 118.22433011025451
error percentage 21.880752690984036
76 108.96016916565863
error percentage 43.36864

198 134.23451957687038
error percentage 32.20478809248971
192 142.31632749169106
error percentage 25.876912764744244
120 119.30066344695504
error percentage 0.5827804608707984
109 77.86505844576833
error percentage 28.564166563515297
149 124.89357871931855
error percentage 16.178806228645264
96 148.02205768506232
error percentage 54.18964342193991
141 122.50623833834014
error percentage 13.116143022453802
84 137.5552502848127
error percentage 63.75625033906274
131 109.99564293588902
error percentage 16.03386035428319
46 107.1871282061198
error percentage 133.01549610026044
129 110.56605960521107
error percentage 14.289876275030181
133 124.31746079085077
error percentage 6.528224969285133
140 127.49112377441476
error percentage 8.934911589703743
195 146.5953810982882
error percentage 24.822881488057337
176 138.1713958290599
error percentage 21.49352509712506
90 120.00330765043184
error percentage 33.33700850047982
194 135.8977363848719
error percentage 29.949620420169122
158 120.7277794

error percentage 31.678553846052225
164 126.89951228726451
error percentage 22.622248605326515
95 80.6659739011878
error percentage 15.088448525065477
93 131.56592322244458
error percentage 41.46873464778987
183 114.08445592352312
error percentage 37.6587672549054
116 100.49664046422684
error percentage 13.364965117045825
174 127.45345824521443
error percentage 26.750886065968718
142 115.74593535783951
error percentage 18.488777917014428
158 125.67183595963196
error percentage 20.46086331668863
80 107.21280918269723
error percentage 34.01601147837154
163 125.73731082558938
error percentage 22.86054550577339
165 146.5953810982882
error percentage 11.154314485885948
76 117.24186484934744
error percentage 54.26561164387821
113 110.98807655323891
error percentage 1.7804632272222018
113 135.69352939538405
error percentage 20.08276937644606
154 110.5403786286336
error percentage 28.220533358030124
119 128.75713061821853
error percentage 8.199269427074391
145 119.8263340133265
error percentag

...

error percentage 140.70855987226966
171 117.42470944054988
error percentage 31.330579274532237
74 123.63010862238579
error percentage 67.06771435457539
147 133.2693701261801
error percentage 9.340564540013542
117 139.4919078248405
error percentage 19.223852841744012
165 115.25176973618137
error percentage 30.1504425841325
72 103.05156641995944
error percentage 43.12717558327701
68 78.19799801551416
error percentage 14.99705590516788
158 139.87536408103574
error percentage 11.471288556306494
131 107.41402050329019
error percentage 18.0045645013052
107 117.4362956898923
error percentage 9.753547373731125
70 99.9028506843042
error percentage 42.71835812043457
66 116.33624655816237
error percentage 76.26704023963997
67 130.22131597805102
error percentage 94.36017310156869
131 141.73045711453958
error percentage 8.191188637053111
91 105.2674468431381
error percentage 15.678513014437476
141 117.29213546400804
error percentage 16.81408832339855
112 86.97711369034332
error percentage 22.341862

error percentage 74.85099218692224
149 131.4527550333129
error percentage 11.776674474286638
70 124.09991231566845
error percentage 77.28558902238349
168 126.89342628116839
error percentage 24.468198642161674
161 117.30484171511362
error percentage 27.139849866389053
131 139.0081084787224
error percentage 6.113059907421673
186 127.26783418923374
error percentage 31.576433231594766
137 121.33934542449083
error percentage 11.431134726649029
92 123.78537288293494
error percentage 34.54931835101623
133 129.00256855280327
error percentage 3.005587554283257
206 133.08997882150945
error percentage 35.3932141643158
143 118.05487181388521
error percentage 17.444145584695654
164 150.39575594244005
error percentage 8.29527076680485
81 130.2941770325426
error percentage 60.85700868215135
144 106.8137456334188
error percentage 25.823787754570276
95 109.38343361552911
error percentage 15.140456437399067
93 102.93893537332309
error percentage 10.687027283143108
134 114.37009361545472
error percentage

error percentage 21.937968945779723
142 139.60475380841038
error percentage 1.686793092668747
63 99.6899222149666
error percentage 58.23797176978825
72 119.74531418127096
error percentage 66.31293636287633
107 111.43647343156132
error percentage 4.146236851926472
92 86.22428448079275
error percentage 6.277951651312232
96 95.50515547764535
error percentage 0.5154630441194316
91 138.82743528319872
error percentage 52.55762119032826
136 130.71823207408178
error percentage 3.883652886704571
66 83.76332768368813
error percentage 26.91413285407293
133 124.59418794917003
error percentage 6.32015943671426
117 136.03794637351163
error percentage 16.27174903718943
111 128.6342466508341
error percentage 15.886708694445122
128 127.51330628789292
error percentage 0.38022946258365886
84 109.84680202259597
error percentage 30.770002407852342
194 119.44605515116406
error percentage 38.42986847878141
102 103.24309446914654
error percentage 1.2187200677907268
58 81.10072425148249
error percentage 39.828

59 91.90224519560309
error percentage 55.766517280683196
88 105.44300397537994
error percentage 19.821595426568113
95 114.97824543779906
error percentage 21.029732039788485
66 109.0761915154675
error percentage 65.26695684161743
157 136.7221682375215
error percentage 12.915816409221982
110 112.32131417577874
error percentage 2.1102856143443076
158 113.10101700750302
error percentage 28.417077843352523
167 136.3998280222215
error percentage 18.32345627411886
172 136.906403041895
error percentage 20.40325404540988
94 136.00489111198374
error percentage 44.68605437445079
66 77.57852725901411
error percentage 17.543223119718345
85 91.16518982230332
error percentage 7.253164496827433
125 125.56441142897353
error percentage 0.45152914317882276
99 137.38891169210962
error percentage 38.776678476878395
81 91.67202953216702
error percentage 13.175345101440762
145 113.70562107381295
error percentage 21.582330293922105
147 125.68732653898289
error percentage 14.498417320419804
180 137.19698549963

64 142.09240246906148
error percentage 122.01937885790856
124 127.23347520135128
error percentage 2.6076412914123246
161 146.8012085540595
error percentage 8.819125121702166
158 142.6751101010324
error percentage 9.699297404409872
127 133.89254843638454
error percentage 5.427203493216177
125 137.79761845524953
error percentage 10.238094764199628
144 125.48565135875394
error percentage 12.857186556420874
125 151.46593603282844
error percentage 21.17274882626275
90 100.35837905694947
error percentage 11.509310063277194
71 141.63595338917384
error percentage 99.48725829461104
103 142.22818603457293
error percentage 38.0856175092941
136 125.00620976595495
error percentage 8.083669289739007
160 144.2010898582504
error percentage 9.874318838593492
100 105.30303496004214
error percentage 5.30303496004214
115 130.7637202736563
error percentage 13.707582846657656
94 114.84984055491184
error percentage 22.180681441395567
151 141.6527308599561
error percentage 6.190244463605226
131 116.8672922540

In [202]:
mean_absolute_error(y_test, pred_test)

29.54143093100904

In [191]:
print('actual value, predicted value, error, percentage in error')
total_error_percentage=0
error_less_10=0
for i in range(6440):
    #if pred[i]!=99:
        error=abs(y_train.iloc[i]['traveltime']-pred_train[i])
        #if error<10:
        #    error_less_10+=1
        #error_percentage=error*100/y_train.iloc[i]['traveltime']
        print(y_train.iloc[i]['traveltime'],pred_train[i], error, error_percentage)
        total_error_percentage+=error_percentage
avg_error_percentage=total_error_percentage/1610
print('average error percentage for total test sample',avg_error_percentage)
print("Number of rows where the error is less than 10", error_less_10)

actual value, predicted value, error, percentage in error
99 131.17585315326923 32.17585315326923 traveltime    35.334377
Name: 17351, dtype: float64
56 100.37410227726386 44.37410227726386 traveltime    35.334377
Name: 17351, dtype: float64
75 85.90711874729774 10.907118747297744 traveltime    35.334377
Name: 17351, dtype: float64
143 134.0094672166503 8.990532783349693 traveltime    35.334377
Name: 17351, dtype: float64
66 111.18057679154781 45.18057679154781 traveltime    35.334377
Name: 17351, dtype: float64
118 136.37583951342074 18.375839513420743 traveltime    35.334377
Name: 17351, dtype: float64
92 85.90711874729774 6.092881252702256 traveltime    35.334377
Name: 17351, dtype: float64
171 142.34833940122397 28.651660598776033 traveltime    35.334377
Name: 17351, dtype: float64
86 114.60292249326365 28.602922493263648 traveltime    35.334377
Name: 17351, dtype: float64
54 77.57852725901411 23.57852725901411 traveltime    35.334377
Name: 17351, dtype: float64
76 87.6479463999982

107 139.40543793241017 32.405437932410166 traveltime    35.334377
Name: 17351, dtype: float64
123 112.55330380086316 10.446696199136838 traveltime    35.334377
Name: 17351, dtype: float64
104 128.39187848039722 24.391878480397224 traveltime    35.334377
Name: 17351, dtype: float64
55 99.0601535573047 44.0601535573047 traveltime    35.334377
Name: 17351, dtype: float64
71 121.73195600790525 50.731956007905254 traveltime    35.334377
Name: 17351, dtype: float64
84 126.87101840074295 42.871018400742955 traveltime    35.334377
Name: 17351, dtype: float64
86 136.73655102985222 50.73655102985222 traveltime    35.334377
Name: 17351, dtype: float64
150 140.3282656204797 9.671734379520302 traveltime    35.334377
Name: 17351, dtype: float64
57 104.97389293323032 47.97389293323032 traveltime    35.334377
Name: 17351, dtype: float64
85 112.29200805416782 27.292008054167823 traveltime    35.334377
Name: 17351, dtype: float64
109 136.29571296948893 27.29571296948893 traveltime    35.334377
Name: 173

76 95.72559481843172 19.725594818431716 traveltime    35.334377
Name: 17351, dtype: float64
192 135.62712611224654 56.37287388775346 traveltime    35.334377
Name: 17351, dtype: float64
103 103.24309446914654 0.24309446914654131 traveltime    35.334377
Name: 17351, dtype: float64
159 117.79911734861524 41.20088265138476 traveltime    35.334377
Name: 17351, dtype: float64
91 116.61818219778169 25.618182197781692 traveltime    35.334377
Name: 17351, dtype: float64
106 121.17815679516923 15.17815679516923 traveltime    35.334377
Name: 17351, dtype: float64
127 127.99093250181892 0.9909325018189179 traveltime    35.334377
Name: 17351, dtype: float64
101 88.6309912729849 12.3690087270151 traveltime    35.334377
Name: 17351, dtype: float64
58 117.96296817366299 59.962968173662986 traveltime    35.334377
Name: 17351, dtype: float64
83 116.88308224000615 33.883082240006146 traveltime    35.334377
Name: 17351, dtype: float64
124 131.51419327976996 7.514193279769955 traveltime    35.334377
Name: 

Name: 17351, dtype: float64
87 140.06609400178564 53.06609400178564 traveltime    35.334377
Name: 17351, dtype: float64
141 123.18996321315551 17.810036786844492 traveltime    35.334377
Name: 17351, dtype: float64
127 121.213494689867 5.786505310132995 traveltime    35.334377
Name: 17351, dtype: float64
83 97.5632796815178 14.563279681517798 traveltime    35.334377
Name: 17351, dtype: float64
99 126.35778232559144 27.357782325591444 traveltime    35.334377
Name: 17351, dtype: float64
184 129.61980360030805 54.380196399691954 traveltime    35.334377
Name: 17351, dtype: float64
138 135.75873640679657 2.2412635932034277 traveltime    35.334377
Name: 17351, dtype: float64
114 138.82716078494772 24.827160784947722 traveltime    35.334377
Name: 17351, dtype: float64
176 146.19572857784286 29.804271422157143 traveltime    35.334377
Name: 17351, dtype: float64
104 100.71694898733628 3.2830510126637193 traveltime    35.334377
Name: 17351, dtype: float64
107 108.4873388126014 1.4873388126013936 

151 92.69740535723524 58.30259464276476 traveltime    35.334377
Name: 17351, dtype: float64
112 134.03889854263278 22.03889854263278 traveltime    35.334377
Name: 17351, dtype: float64
68 107.65673356982448 39.65673356982448 traveltime    35.334377
Name: 17351, dtype: float64
105 107.1871282061198 2.1871282061197945 traveltime    35.334377
Name: 17351, dtype: float64
103 135.94457521405886 32.94457521405886 traveltime    35.334377
Name: 17351, dtype: float64
89 139.7314729894728 50.73147298947279 traveltime    35.334377
Name: 17351, dtype: float64
170 132.12854131313114 37.87145868686886 traveltime    35.334377
Name: 17351, dtype: float64
172 119.08692764572532 52.913072354274675 traveltime    35.334377
Name: 17351, dtype: float64
97 125.40714609413182 28.407146094131818 traveltime    35.334377
Name: 17351, dtype: float64
141 106.09145228649679 34.90854771350321 traveltime    35.334377
Name: 17351, dtype: float64
149 113.74986425042076 35.25013574957924 traveltime    35.334377
Name: 17

...

Name: 17351, dtype: float64
102 112.63121118331445 10.631211183314448 traveltime    35.334377
Name: 17351, dtype: float64
116 146.14382313188813 30.143823131888126 traveltime    35.334377
Name: 17351, dtype: float64
113 117.67316361708286 4.673163617082864 traveltime    35.334377
Name: 17351, dtype: float64
86 131.36491033074685 45.36491033074685 traveltime    35.334377
Name: 17351, dtype: float64
176 131.48836057368743 44.51163942631257 traveltime    35.334377
Name: 17351, dtype: float64
84 127.65523467645878 43.655234676458775 traveltime    35.334377
Name: 17351, dtype: float64
116 129.73034467591867 13.730344675918673 traveltime    35.334377
Name: 17351, dtype: float64
104 122.13809018428869 18.138090184288686 traveltime    35.334377
Name: 17351, dtype: float64
99 80.60965837786964 18.390341622130364 traveltime    35.334377
Name: 17351, dtype: float64
87 110.57101317537435 23.571013175374347 traveltime    35.334377
Name: 17351, dtype: float64
165 122.46821003911798 42.53178996088202

127 150.12909326109812 23.12909326109812 traveltime    35.334377
Name: 17351, dtype: float64
74 117.31781644058547 43.317816440585474 traveltime    35.334377
Name: 17351, dtype: float64
137 117.62926533657249 19.370734663427513 traveltime    35.334377
Name: 17351, dtype: float64
85 116.87224582420328 31.872245824203276 traveltime    35.334377
Name: 17351, dtype: float64
174 121.65906530624923 52.34093469375077 traveltime    35.334377
Name: 17351, dtype: float64
218 139.4091327741728 78.5908672258272 traveltime    35.334377
Name: 17351, dtype: float64
78 90.80161570581676 12.801615705816758 traveltime    35.334377
Name: 17351, dtype: float64
82 119.9006831038529 37.9006831038529 traveltime    35.334377
Name: 17351, dtype: float64
50 110.64491924262167 60.64491924262167 traveltime    35.334377
Name: 17351, dtype: float64
49 109.58039853069262 60.58039853069262 traveltime    35.334377
Name: 17351, dtype: float64
147 115.21706585220149 31.782934147798514 traveltime    35.334377
Name: 17351

161 134.82208046551463 26.17791953448537 traveltime    35.334377
Name: 17351, dtype: float64
67 113.10101700750302 46.10101700750302 traveltime    35.334377
Name: 17351, dtype: float64
107 110.16602281114389 3.1660228111438897 traveltime    35.334377
Name: 17351, dtype: float64
191 133.18631628078236 57.813683719217636 traveltime    35.334377
Name: 17351, dtype: float64
142 134.69096865675803 7.309031343241969 traveltime    35.334377
Name: 17351, dtype: float64
150 120.48195798520938 29.518042014790623 traveltime    35.334377
Name: 17351, dtype: float64
78 109.66005766195674 31.660057661956742 traveltime    35.334377
Name: 17351, dtype: float64
87 128.97166285145263 41.971662851452635 traveltime    35.334377
Name: 17351, dtype: float64
185 134.94227045388863 50.057729546111375 traveltime    35.334377
Name: 17351, dtype: float64
87 124.00304806467447 37.00304806467447 traveltime    35.334377
Name: 17351, dtype: float64
100 122.88955323050492 22.889553230504916 traveltime    35.334377
Na

249 129.14837268669362 119.85162731330638 traveltime    35.334377
Name: 17351, dtype: float64
102 120.86218579448231 18.862185794482315 traveltime    35.334377
Name: 17351, dtype: float64
67 78.44398751520107 11.443987515201073 traveltime    35.334377
Name: 17351, dtype: float64
136 140.49272091831884 4.492720918318838 traveltime    35.334377
Name: 17351, dtype: float64
136 121.74149530053316 14.25850469946684 traveltime    35.334377
Name: 17351, dtype: float64
95 118.9266149263105 23.926614926310506 traveltime    35.334377
Name: 17351, dtype: float64
64 109.54603954281015 45.54603954281015 traveltime    35.334377
Name: 17351, dtype: float64
97 114.35346975966434 17.353469759664335 traveltime    35.334377
Name: 17351, dtype: float64
209 126.18768817858061 82.81231182141939 traveltime    35.334377
Name: 17351, dtype: float64
94 129.8686193748059 35.86861937480589 traveltime    35.334377
Name: 17351, dtype: float64
126 125.63063409261831 0.3693659073816917 traveltime    35.334377
Name: 1

122 107.85796091577474 14.14203908422526 traveltime    35.334377
Name: 17351, dtype: float64
147 140.1398869684485 6.860113031551492 traveltime    35.334377
Name: 17351, dtype: float64
166 132.00374680476017 33.99625319523983 traveltime    35.334377
Name: 17351, dtype: float64
119 141.98171086866205 22.98171086866205 traveltime    35.334377
Name: 17351, dtype: float64
135 129.73034467591867 5.269655324081327 traveltime    35.334377
Name: 17351, dtype: float64
99 107.50981632553912 8.50981632553912 traveltime    35.334377
Name: 17351, dtype: float64
110 107.4158331452215 2.584166854778502 traveltime    35.334377
Name: 17351, dtype: float64
214 146.07861612047563 67.92138387952437 traveltime    35.334377
Name: 17351, dtype: float64
130 132.74711743712407 2.747117437124075 traveltime    35.334377
Name: 17351, dtype: float64
303 99.56495754439509 203.4350424556049 traveltime    35.334377
Name: 17351, dtype: float64
52 120.40974959983895 68.40974959983895 traveltime    35.334377
Name: 17351

In [62]:
pred_test=nn.predict(x_test)


In [63]:
pred_test.shape

(1610,)

In [64]:
y_test.head(10)

3296     50
616     128
3456     68
6390     78
2592     61
923      98
624      73
8008    129
497      51
7360    142
dtype: int64

In [65]:
y_test.iloc[1]

128

In [66]:
print('actual value, predicted value, error, percentage in error')
total_error_percentage=0
error_less_10=0
for i in range(1610):
    #if pred[i]!=99:
        if error<10:
            error_less_10+=1
        error=abs(y_test.iloc[i]-pred_test[i])
        error_percentage=error*100/y_test.iloc[i]
        print(y_test.iloc[i],pred_test[i], error, error_percentage)
        total_error_percentage+=error_percentage
avg_error_percentage=total_error_percentage/1610
print('average error percentage for total test sample',avg_error_percentage)
print("Number of rows where the error is less than 10", error_less_10)

actual value, predicted value, error, percentage in error
50 99 49 98.0
128 83 45 35.15625
68 105 37 54.411764705882355
78 83 5 6.410256410256411
61 90 29 47.540983606557376
98 105 7 7.142857142857143
73 83 10 13.698630136986301
129 105 24 18.6046511627907
51 105 54 105.88235294117646
142 105 37 26.056338028169016
220 105 115 52.27272727272727
140 90 50 35.714285714285715
85 99 14 16.470588235294116
101 83 18 17.821782178217823
127 105 22 17.322834645669293
186 83 103 55.376344086021504
125 90 35 28.0
63 105 42 66.66666666666667
83 105 22 26.50602409638554
150 105 45 30.0
229 105 124 54.148471615720524
199 105 94 47.23618090452261
90 123 33 36.666666666666664
166 99 67 40.36144578313253
114 83 31 27.19298245614035
175 99 76 43.42857142857143
117 83 34 29.05982905982906
125 90 35 28.0
116 90 26 22.413793103448278
145 105 40 27.586206896551722
65 105 40 61.53846153846154
152 90 62 40.78947368421053
142 90 52 36.61971830985915
171 123 48 28.07017543859649
85 123 38 44.705882352941174
136 

119 90 29 24.369747899159663
80 105 25 31.25
137 105 32 23.357664233576642
174 99 75 43.10344827586207
143 105 38 26.573426573426573
87 89 2 2.2988505747126435
126 105 21 16.666666666666668
100 90 10 10.0
67 99 32 47.76119402985075
94 83 11 11.702127659574469
55 99 44 80.0
139 105 34 24.46043165467626
75 89 14 18.666666666666668
103 105 2 1.941747572815534
139 83 56 40.28776978417266
115 105 10 8.695652173913043
123 83 40 32.520325203252035
65 105 40 61.53846153846154
93 90 3 3.225806451612903
116 83 33 28.448275862068964
118 89 29 24.576271186440678
94 105 11 11.702127659574469
181 123 58 32.04419889502763
141 105 36 25.53191489361702
77 105 28 36.36363636363637
158 105 53 33.54430379746835
207 90 117 56.52173913043478
50 83 33 66.0
72 105 33 45.833333333333336
91 105 14 15.384615384615385
171 83 88 51.461988304093566
80 105 25 31.25
91 83 8 8.791208791208792
188 83 105 55.851063829787236
109 83 26 23.853211009174313
94 105 11 11.702127659574469
81 105 24 29.62962962962963
105 90 15 1

171 83 88 51.461988304093566
108 105 3 2.7777777777777777
137 90 47 34.306569343065696
117 105 12 10.256410256410257
69 99 30 43.47826086956522
163 105 58 35.58282208588957
139 105 34 24.46043165467626
152 83 69 45.39473684210526
84 105 21 25.0
142 83 59 41.54929577464789
79 90 11 13.924050632911392
207 90 117 56.52173913043478
87 105 18 20.689655172413794
230 99 131 56.95652173913044
97 105 8 8.24742268041237
149 99 50 33.557046979865774
61 105 44 72.1311475409836
180 90 90 50.0
103 123 20 19.41747572815534
59 83 24 40.67796610169491
141 123 18 12.76595744680851
55 83 28 50.90909090909091
64 90 26 40.625
155 105 50 32.25806451612903
154 123 31 20.12987012987013
152 90 62 40.78947368421053
79 90 11 13.924050632911392
103 83 20 19.41747572815534
76 90 14 18.42105263157895
102 90 12 11.764705882352942
110 105 5 4.545454545454546
131 83 48 36.6412213740458
88 83 5 5.681818181818182
142 90 52 36.61971830985915
68 105 37 54.411764705882355
157 90 67 42.67515923566879
93 105 12 12.9032258064

100 105 5 5.0
119 83 36 30.252100840336134
90 105 15 16.666666666666668
71 105 34 47.88732394366197
63 123 60 95.23809523809524
98 105 7 7.142857142857143
191 123 68 35.602094240837694
146 123 23 15.753424657534246
122 99 23 18.852459016393443
98 90 8 8.16326530612245
75 90 15 20.0
56 83 27 48.214285714285715
154 123 31 20.12987012987013
135 105 30 22.22222222222222
114 90 24 21.05263157894737
134 105 29 21.64179104477612
86 105 19 22.093023255813954
75 105 30 40.0
82 105 23 28.048780487804876
64 105 41 64.0625
100 105 5 5.0
109 90 19 17.431192660550458
80 105 25 31.25
98 83 15 15.306122448979592
145 123 22 15.172413793103448
129 105 24 18.6046511627907
108 99 9 8.333333333333334
112 123 11 9.821428571428571
56 105 49 87.5
175 105 70 40.0
168 105 63 37.5
57 123 66 115.78947368421052
86 99 13 15.116279069767442
137 90 47 34.306569343065696
142 105 37 26.056338028169016
137 105 32 23.357664233576642
60 83 23 38.333333333333336
102 105 3 2.9411764705882355
115 83 32 27.82608695652174
70 8

Trying by normalising data 


In [28]:
df_norm=(df - df.min()) / (df.max() - df.min())

In [29]:
x_train_norm, x_test_norm, y_train_norm, y_test_norm=train_test_split(df_norm, df_traveltime, test_size=0.2, random_state=4)

In [30]:
x_train_norm.head(10)

,day,hour,rain,temp
571,0.166667,NaN,0.00,0.310734
6270,0.666667,NaN,0.00,0.638418
2352,0.333333,NaN,0.25,0.666667
6081,0.666667,NaN,0.00,0.553672
3035,0.333333,NaN,0.00,0.655367
2210,0.000000,NaN,0.00,0.186441
477,0.000000,NaN,0.00,0.570621
973,0.500000,NaN,0.00,0.440678
6862,0.333333,NaN,0.00,0.604520
2483,0.500000,NaN,0.00,0.553672


In [34]:
df.head(10)

,day,hour,rain,temp
0,6,0,0.0,5.1
1,6,0,0.0,5.1
2,6,0,0.0,5.1
3,6,0,0.0,5.1
4,6,0,0.0,5.1
5,6,0,0.0,5.1
6,6,0,0.0,5.1
7,6,0,0.0,5.1
8,6,0,0.0,5.1
9,6,0,0.0,5.1


In [35]:
df['hour'].unique()

array([0])

In [73]:
nn_norm=MLPClassifier(activation='identity',solver='sgd',hidden_layer_sizes=(100,50),random_state=1)

In [75]:
nn_norm.fit(x_train_norm, y_train_norm)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [3]:
def nonlin(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

In [5]:
X = np.array([ [0,0,1],[0,1,1],[1,0,1],[1,1,1] ])

y = np.array([[0,0,1,1]]).T


In [6]:
np.random.seed(1)

In [7]:
syn0 = 2*np.random.random((3,1)) - 1

In [9]:
for iter in range(10000):
    l0 = X
    l1 = nonlin(np.dot(l0,syn0))
    l1_error = y - l1
    l1_delta = l1_error * nonlin(l1,True)
    syn0 += np.dot(l0.T,l1_delta)
    

In [10]:
print("Output After Training:")
print(l1)

Output After Training:
[[0.00966449]
 [0.00786506]
 [0.99358898]
 [0.99211957]]


In [3]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import rmsprop

import matplotlib.pyplot as plt
from scipy.misc import toimage

from sklearn import metrics

import pandas as pd
import numpy as np

from IPython.display import display, HTML, Image
from IPython.display import SVG

from PIL import Image

import os
from random import randint

/home/student/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [42]:
model = Sequential()
model.add(Dense(input_dim=4, units=128))
model.add(Activation("relu"))
model.add(Dense(units=64))
model.add(Activation("relu"))
#model.add(Dense(units=256))
#model.add(Activation("relu"))
model.add(Dense(units=1))
model.add(Activation("softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               640       
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
activation_9 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
_________________________________________________________________
activation_10 (Activation)   (None, 1)                 0         
Total params: 8,961
Trainable params: 8,961
Non-trainable params: 0
_________________________________________________________________


In [5]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

In [43]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [44]:
x_train.shape


(6440, 4)

In [45]:
y_train.shape

(6440,)

In [46]:
model.fit(x_train, y_train, \
          epochs=20, batch_size=6440, verbose=1)

ValueError: You are passing a target array of shape (6440, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.